### 09_VAE_fashion

In [1]:
import os,sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets,layers,models,optimizers,Sequential,metrics,losses,utils
import matplotlib.pyplot as plt
from PIL import Image

from tensorflow.keras.preprocessing import sequence

print(sys.version)
print(tf.test.is_gpu_available())
print(tf.__version__)

3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
False
2.0.0


In [2]:
#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 
class_names = ["上衣","褲子","套衫","連衣裙","外套","涼鞋","襯衫","運動鞋","袋子","踝靴"]

def data_normalize(train, test, scale=1):
    (x_train,y_train) = train
    (x_test,y_test) = test
    train_nums = x_train.shape[0] // scale
    test_nums = x_test.shape[0] // scale
    x_train, y_train = x_train[:train_nums], y_train[:train_nums]
    x_test, y_test = x_test[:train_nums], y_test[:train_nums]
    x_train = x_train.astype(np.float32) / 255.
    x_test = x_test.astype(np.float32) / 255.
    
    
    x_train = tf.reshape(x_train, shape=(-1,28*28))
    x_test = tf.reshape(x_test, shape=(-1,28*28))
    #y_train = utils.to_categorical(y_train)#.astype(np.int32)
    #y_test = utils.to_categorical(y_test)
    print(x_train.shape,y_train.shape)
    print(x_test.shape,y_test.shape)
    
    return (x_train,y_train),(x_test,y_test)

(x_train,y_train),(x_test,y_test) = datasets.fashion_mnist.load_data()
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)
(x_train,y_train),(x_test,y_test) = data_normalize((x_train,y_train),(x_test,y_test),scale=10)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
(6000, 784) (6000,)
(6000, 784) (6000,)


In [3]:
dim_reduce = 10

class VAE(models.Model):
    def __init__(self):
        super(VAE,self).__init__()
        #encoder
        self.fc_layer_1 = layers.Dense(128)
        self.fc_layer_2 = layers.Dense(dim_reduce)
        self.fc_layer_3 = layers.Dense(dim_reduce)   
        self.fc_layer_4 = layers.Dense(128)
        self.fc_layer_5 = layers.Dense(784)

    def model_encoder(self, x):
        h = tf.nn.relu(self.fc_layer_1(x))
        mean_fc = self.fc_layer_2(h)
        var_fc = self.fc_layer_3(h)
        return mean_fc,var_fc

    def model_decoder(self, z):
        out = tf.nn.relu(self.fc_layer_4(z))
        out = self.fc_layer_5(out)
        return out

    def reparameter(self,mean_x,var_x):
        eps = tf.random.normal(var_x.shape)
        std = tf.exp(var_x)**0.5
        z = mean_x + std*eps
        return z
  
    def call(self, inputs, training=None):
        mean_x,var_x = self.model_encoder(inputs)
        z = self.reparameter(mean_x,var_x)
        x = self.model_decoder(z)
        return x,mean_x,var_x

In [4]:
model = VAE()
model.build(input_shape=(4,784))
optimizer = optimizers.Adam(lr=1e-3)
model.summary()

Model: "vae"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
_________________________________________________________________
dense_2 (Dense)              multiple                  1290      
_________________________________________________________________
dense_3 (Dense)              multiple                  1408      
_________________________________________________________________
dense_4 (Dense)              multiple                  101136    
Total params: 205,604
Trainable params: 205,604
Non-trainable params: 0
_________________________________________________________________


In [13]:
# FAIL
# batch_size=128
# epochs=6
# optimizer = optimizers.Adam(lr=1e-3)
# model.compile(loss="sigmoid_cross_entropy_with_logits",optimizer=optimizer)
# history = model.fit(x_train,x_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test,x_test))

In [11]:
dim_reduce = 10
batch_num = 128
lr = 1e-3

def feature_scale(x):
    x = tf.cast(x,dtype=tf.float32)/255.
    return x
(x,y),(x_test,y_test) = datasets.fashion_mnist.load_data()
data = tf.data.Dataset.from_tensor_slices(x)
data = data.map(feature_scale).shuffle(10000).batch(batch_num)

data_test = tf.data.Dataset.from_tensor_slices(x_test)
data_test = data_test.map(feature_scale).batch(batch_num)

data_iter = iter(data)
samples = next(data_iter)
print(samples[0].shape,samples[1].shape)

model = VAE()
model.build(input_shape=(4,784))
optimizer = optimizers.Adam(lr=lr)
model.summary()

optimizer = optimizers.Adam(lr=lr)


(28, 28) (28, 28)
Model: "vae_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              multiple                  100480    
_________________________________________________________________
dense_6 (Dense)              multiple                  1290      
_________________________________________________________________
dense_7 (Dense)              multiple                  1290      
_________________________________________________________________
dense_8 (Dense)              multiple                  1408      
_________________________________________________________________
dense_9 (Dense)              multiple                  101136    
Total params: 205,604
Trainable params: 205,604
Non-trainable params: 0
_________________________________________________________________


In [12]:
for i in range(3):
    for step,x in enumerate(data):
        x = tf.reshape(x,[-1,784])
        with tf.GradientTape() as tape:
            logits,mean_x,var_x = model(x)
            loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=x,logits=logits)
            loss = tf.reduce_sum(loss)/x.shape[0]
            kl_div = -0.5*(var_x+1-mean_x**2-tf.exp(var_x))
            kl_div = tf.reduce_sum(kl_div)/x.shape[0]

            loss = loss + 1.*kl_div
        grads = tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(zip(grads,model.trainable_variables))

        if step %100==0:
            print(i,step,'loss:',float(loss),'kl_div:',float(kl_div))

0 0 loss: 551.7018432617188 kl_div: 4.142721176147461
0 100 loss: 313.9952087402344 kl_div: 14.881878852844238
0 200 loss: 288.0704650878906 kl_div: 16.23518943786621
0 300 loss: 261.48370361328125 kl_div: 15.778911590576172
0 400 loss: 254.17156982421875 kl_div: 14.694847106933594
1 0 loss: 279.21954345703125 kl_div: 15.134773254394531
1 100 loss: 259.7195129394531 kl_div: 13.590936660766602
1 200 loss: 265.6165466308594 kl_div: 14.831949234008789
1 300 loss: 261.96588134765625 kl_div: 15.054460525512695
1 400 loss: 257.9376525878906 kl_div: 13.888978958129883
2 0 loss: 253.5751495361328 kl_div: 14.487897872924805
2 100 loss: 251.00926208496094 kl_div: 14.010517120361328
2 200 loss: 263.02459716796875 kl_div: 13.882869720458984
2 300 loss: 258.3230895996094 kl_div: 14.516339302062988
2 400 loss: 254.13148498535156 kl_div: 14.334625244140625


In [15]:
def save_img(imgs,names):
    img_new = Image.new('L',(280,280))
    index = 0
    for i in range(0,280,80):
        for j in range(0,280,80):
            img = imgs[index]
            img = Image.fromarray(img,mode='L')
            img_new.paste(img,(i,j))
            index+=1
    img_new.save(names)

x = next(iter(data_test))
val_x = tf.reshape(x,[-1,784])
logits,_,_ = model(val_x)
x_hat = tf.sigmoid(logits)
x_hat = tf.reshape(x_hat,[-1,28,28])
x_hat = x_hat.numpy()*255
x_hat = x_hat.astype(np.uint8)
save_img(x_hat,'imgs/VAE_img_%d.png'%i)

![efc](./imgs/VAE_img_2.png)